In [106]:
import pandas as pd
import numpy as np
import re
data = pd.read_csv('techscene.csv')

# Andmete puhastamine

In [107]:
for column in data.columns:#teeme kõik väiketähtedeks
  data[column]=data[column].str.lower()
#job_name
#eemaldame (m/f/d),(m/f/x),| SEB, Tallinn,
data['job_name']=data['job_name'].str.replace('(m/f/d)', '').replace('(m/f/x)','').replace('| SEB, Tallinn','').replace('\"','')



In [115]:
data['domain'].unique()

array(['software-development', nan, 'front-end', 'back-end', 'mobile',
       'analytics', 'full-stack', 'other', 'devops-sre', 'data-science',
       'cyber-security', 'ai-ml', 'qa', 'sales', 'finance', 'marketing',
       'design', 'hr', 'customer-support', 'management'], dtype=object)

In [118]:
for index, row in data[data['domain'].isna()].iterrows():#lisame valdkonna kõikjale, kus domain on NaN
  name=row['job_name']
  if re.search('software engineer|tarkvaraarendaja',name):
    print('replaced ',str(index))
    data.loc[index,'domain']='software-development'
  elif re.search('front-end|frontend|react',name):
    data.loc[index,'domain']='front-end'
  elif re.search('back-end|backend',name):
    data.loc[index,'domain']='back-end'
  elif re.search('android|ios',name):
    data.loc[index,'domain']='mobile'
  elif re.search('devops',name):
    data.loc[index,'domain']='devops-sre'
  elif re.search('data scientist',name):
    data.loc[index,'domain']='data-science'

  

In [119]:
data[data['domain'].isna()]

,company_name,job_name,job_link,date_added,domain
6,nexd,"full-stack developer (php, vue.js)",https://www.nexd.com/career/full-stack-developer/,2024-03-26 00:00:00,NaN
27,nexd,"full-stack developer (php, vue.js)",https://www.nexd.com/career/full-stack-developer/,2024-03-26 00:00:00,NaN
29,novater solutions,full-stack developer,https://novater.com/careers/full-stack-developer/,2024-03-26 00:00:00,NaN
30,seb,full-stack software developer for baltic inter...,https://jobs.eu.lever.co/seb/2b1135be-e2e0-4f7...,2024-03-26 00:00:00,NaN
31,creditstar,full stack developer,https://apply.workable.com/j/9c81ddebd2,2024-03-26 00:00:00,NaN
...,...,...,...,...,...
737,luminor,senior compliance officer (assurance unit),https://luminorbank.teamtailor.com/jobs/354120...,2024-04-02 00:00:00,NaN
738,cybernetica,post-quantum cryptography engineer,https://cyber.teamdash.com/p/job/iqn79wzx/post...,2024-04-02 00:00:00,NaN
739,cybernetica,researcher,https://cyber.teamdash.com/p/job/pagseczx/rese...,2024-04-02 00:00:00,NaN
740,cybernetica,data architect,https://cyber.teamdash.com/p/job/knvredgr/data...,2024-04-02 00:00:00,NaN


In [ ]:
"""
Groups data by company name, job name, and job link. 
Sorts the data by date added and creates blocks of consecutive job postings.
"""

def process_job_data(data):
    data['date_added'] = pd.to_datetime(data['date_added']).dt.date

    # Normalize text for comparison
    data['job_name'] = data['job_name'].str.lower()
    data['company_name'] = data['company_name'].str.lower()

    # Remove exact duplicates
    data = data.drop_duplicates()

    # Create an empty DataFrame with specified columns
    new_data = pd.DataFrame(columns=["company_name", "job_name", "domain", "first_appeared", "last_appeared", "job_link"])

    grouped = data.groupby(["company_name", "job_name", "job_link"])
    
    for (company_name, job_name, job_link), group in grouped:
        group = group.sort_values('date_added')

        start_date = None
        prev_date = None

        # Get domain if exists in the group
        domain = group['domain'].dropna().iloc[0] if not group['domain'].dropna().empty else None

        for current_date in group['date_added']:
            if start_date is None:
                start_date = current_date
                prev_date = current_date
            elif (current_date - prev_date).days > 1:
                # Special case: check only for 2024-09-06
                if (prev_date == pd.to_datetime('2024-09-05').date() and 
                    current_date == pd.to_datetime('2024-09-07').date()):
                    prev_date = current_date  # Assume continuity
                else:
                    # Save current block if gap is not special case
                    new_data = pd.concat([new_data, pd.DataFrame({
                        "company_name": [company_name],
                        "job_name": [job_name],
                        "domain": [domain],
                        "first_appeared": [start_date],
                        "last_appeared": [prev_date],
                        "job_link": [job_link]
                    })], ignore_index=True)
                    start_date = current_date

                start_date = current_date  # Start a new block

            prev_date = current_date

        # Save the last block
        new_data = pd.concat([new_data, pd.DataFrame({
            "company_name": [company_name],
            "job_name": [job_name],
            "domain": [domain],
            "first_appeared": [start_date],
            "last_appeared": [prev_date],
            "job_link": [job_link]
        })], ignore_index=True)

    return new_data

# Example usage:
# data = pd.read_csv('input.csv')
processed_data = process_job_data(data)
processed_data.to_csv('new_data!!!.csv', index=False)


In [ ]:
"""
Groups data by company name, job name, and job link.
Finds the first apperance and last apperance dates.
"""

def process_job_data(data):
    data['date_added'] = pd.to_datetime(data['date_added']).dt.date

    # Create an empty DataFrame with specified columns
    new_data = pd.DataFrame(columns=["company_name", "job_name", "domain", "first_appeared", "last_appeared", "job_link"])
    
    # Group by unique identifying columns
    grouped = data.groupby(["company_name", "job_name", "job_link"])
    
    # Iterate through each unique group
    for (company_name, job_name, job_link), group in grouped:
        # Find the first and last appearance dates
        first_appeared = group['date_added'].min()
        last_appeared = group['date_added'].max()
        

        # Get domain if exists in the group
        domain = group['domain'].dropna().iloc[0] if not group['domain'].dropna().empty else None

        # Create a new row for this unique entry
        new_row = pd.DataFrame({
            "company_name": [company_name],
            "job_name": [job_name],
            "domain": [domain],
            "first_appeared": [first_appeared],
            "last_appeared": [last_appeared],
            "job_link": [job_link]
        })
        
        # Append to new_data
        new_data = pd.concat([new_data, new_row], ignore_index=True)
    
    return new_data

# Example usage
# data = pd.read_csv('your_input_file.csv')  # Load your input data
nd = data.copy()
new_data = process_job_data(data)
new_data.to_csv('new_data!!!!!!!.csv', index=False)  # Save processed data

In [ ]:
"""
Checks if first data has the same links as second data.
"""
def chek_links(data, data2):

    links1 = data["job_link"].unique()
    links2 = data2["job_link"].unique()
    for link in links1:
        if link not in links2:
            print(link)